### Question 2

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import nltk
import sklearn
import pandas as pd
import math
import operator
import re
from sklearn import decomposition, feature_selection, metrics, model_selection

#nltk.download('all')

In [8]:
wine_train_f = open('Wine/wine_train.csv','r')
wine_test_f = open('Wine/wine_test.csv','r')
wine_train_lines = []
wine_test_lines = []


for line_num, line in enumerate(wine_train_f):
    if line_num == 0:
        line = line.replace('"','')
        line = line.replace('\n','')
        wine_train_lines.append(line.split(';'))
    else:
        n_line = line.split(';')
        n_line = [float(x) for x in n_line]
        wine_train_lines.append(n_line)
        
for line_num, line in enumerate(wine_test_f):
    if line_num == 0:
        line = line.replace('"','')
        wine_test_lines.append(line.split(';'))
    else:
        n_line = line.split(';')
        n_line = [float(x) for x in n_line]
        wine_test_lines.append(n_line)
        
w_train = pd.DataFrame(wine_train_lines[1:], columns=wine_train_lines[0])
w_test = pd.DataFrame(wine_test_lines[1:], columns=wine_test_lines[0])

train_array = w_train.values
train_xs = [x[:-1] for x in train_array]
train_ys = [y[-1:] for y in train_array]
train_ys = np.reshape(train_ys, len(train_ys))

test_array = w_test.values
test_xs = [x[:-1] for x in test_array]
test_ys = [y[-1:] for y in test_array]
test_ys = np.reshape(test_ys, len(test_ys))

w_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6.0


In [9]:
lin_regress = sklearn.linear_model.LinearRegression()
lin_regress.fit(train_xs, train_ys)
preds_lin = lin_regress.predict(test_xs)

print(math.sqrt(sklearn.metrics.mean_squared_error(y_true=test_ys, y_pred=preds_lin)))

0.7743778617318132


In [10]:
svr_model = sklearn.svm.SVR(gamma=0.23, kernel='rbf')
svr_model.fit(train_xs, train_ys)
preds_svr = svr_model.predict(test_xs)

print(math.sqrt(sklearn.metrics.mean_squared_error(test_ys, preds_svr)))

0.7263170196336258


### Question 3

In [5]:
hateval = pd.read_csv('Hateval/hateval.tsv', sep='\t')
hateval.index = hateval['id']
hateval = hateval.drop(['id'],axis=1)
hateval.head()

x = hateval.values
pos_hat = [item[0] for item in x if item[1]==0]
neg_hat = [item[0] for item in x if item[1]==1]

In [6]:
def get_list_tokens(string):
    sentence_split=nltk.tokenize.sent_tokenize(string)
    list_tokens=[]
    for sentence in sentence_split:
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
        for token in list_tokens_sentence:
            list_tokens.append(lemmatizer.lemmatize(token).lower())
            
    return list_tokens


lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords=set(nltk.corpus.stopwords.words('english'))
# We can add more words to the stopword list, like punctuation marks
stopwords.add(".")
stopwords.add(",")
stopwords.add("#")
stopwords.add("@")
stopwords.add(":")
stopwords.add("--")
stopwords.add("``")
stopwords.add("!")
stopwords.add("?")
stopwords.add("...")
stopwords.add("&")
stopwords.add("-")
stopwords.add(";")
stopwords.add("'")
stopwords.add("#")
stopwords.add("’")
    
    
def get_vector_text(list_vocab,string):
    vector_text=np.zeros(len(list_vocab))
    list_tokens_string=get_list_tokens(string)
    for i, word in enumerate(list_vocab):
        if word in list_tokens_string:
            vector_text[i]=list_tokens_string.count(word)
    return vector_text

def get_vocabulary(training_set, num_features): # Function to retrieve vocabulary
    dict_word_frequency={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: continue
            if word not in dict_word_frequency: dict_word_frequency[word]=1
            else: dict_word_frequency[word]+=1
    sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
    vocabulary=[]
    for word,frequency in sorted_list:
        vocabulary.append(word)
    return vocabulary

def train_svm_classifier(training_set, vocabulary): # Function for training our svm classifier
    X_train=[]
    Y_train=[]
    for instance in training_set:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_train.append(vector_instance)
        Y_train.append(instance[1])
    # Finally, we train the SVM classifier 
    svm_clf=sklearn.svm.SVC(kernel="rbf",gamma='scale')
    svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))
    return svm_clf

In [7]:
pos_train, pos_test = sklearn.model_selection.train_test_split(pos_hat, train_size=0.7)
neg_train, neg_test = sklearn.model_selection.train_test_split(neg_hat, train_size=0.7)

In [53]:
X_train = []
Y_train = []
Xvec_train = []

X_test = []
Y_test = []
Xvec_test = []

for pos_tweet in pos_train:
    #vec = get_vector_text(vocabulary, pos_tweet)
    X_train.append(pos_tweet)
    Y_train.append(0)
for neg_tweet in neg_train:
    X_train.append(neg_tweet)
    Y_train.append(1)
    
vocabulary = get_vocabulary(zip(X_train, Y_train), 1000)

for pos_tweet in pos_train:
    vec = get_vector_text(vocabulary, pos_tweet)
    Xvec_train.append(vec)
for neg_tweet in neg_train:
    vec = get_vector_text(vocabulary, neg_tweet)
    Xvec_train.append(vec)
    
for pos_tweet in pos_test:
    vec = get_vector_text(vocabulary, pos_tweet)
    X_test.append(pos_tweet)
    Xvec_test.append(vec)
    Y_test.append(0)
for neg_tweet in neg_test:
    vec = get_vector_text(vocabulary, neg_tweet)
    X_test.append(pos_tweet)
    Xvec_test.append(vec)
    Y_test.append(1)
    
scaler = sklearn.preprocessing.Normalizer()
std_x = scaler.fit_transform(Xvec_train)
std_x_test = scaler.transform(Xvec_test)
    
svm_clf=train_svm_classifier(zip(X_train, Y_train), vocabulary)

svm_clf_std = sklearn.svm.SVC(kernel='rbf', gamma='scale')
svm_clf_std.fit(std_x, Y_train)

chi_x = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=500).fit(Xvec_train, Y_train)
Xvec_train_chi = chi_x.transform(Xvec_train)
Xvec_test_chi = chi_x.transform(Xvec_test)

svm_clf_chi = sklearn.svm.SVC(kernel='rbf', gamma='scale')
svm_clf_chi.fit(Xvec_train_chi, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [54]:
preds = svm_clf.predict(Xvec_test)
preds_std = svm_clf_std.predict(std_x_test)
preds_chi = svm_clf_chi.predict(Xvec_test_chi)
print(sklearn.metrics.classification_report(Y_test, preds))
print(sklearn.metrics.classification_report(Y_test, preds_std))
print(sklearn.metrics.classification_report(Y_test, preds_chi))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82      1566
           1       0.79      0.67      0.72      1135

    accuracy                           0.78      2701
   macro avg       0.79      0.77      0.77      2701
weighted avg       0.79      0.78      0.78      2701

              precision    recall  f1-score   support

           0       0.79      0.87      0.83      1566
           1       0.79      0.67      0.73      1135

    accuracy                           0.79      2701
   macro avg       0.79      0.77      0.78      2701
weighted avg       0.79      0.79      0.78      2701

              precision    recall  f1-score   support

           0       0.79      0.86      0.83      1566
           1       0.78      0.68      0.73      1135

    accuracy                           0.79      2701
   macro avg       0.79      0.77      0.78      2701
weighted avg       0.79      0.79      0.79      2701



In [21]:
s = "Hello world. My name is Cai. Nice to meet you"
x = nltk.tokenize.sent_tokenize(s)
print(x)
y = []
yy = []
for z in x:
    y = nltk.tokenize.word_tokenize(z)
    l = nltk.stem.WordNetLemmatizer()
    for j in y:
        yy.append(l.lemmatize(j).lower())
        
print(yy)        

['Hello world.', 'My name is Cai.', 'Nice to meet you']
['hello', 'world', '.', 'my', 'name', 'is', 'cai', '.', 'nice', 'to', 'meet', 'you']


### Question 3 with K-fold

In [48]:
import random

dataset_full = []
for pos_tweet in pos_hat:
    dataset_full.append((pos_tweet, 0))
for neg_tweet in neg_hat:
    dataset_full.append((neg_tweet, 1))
    
kf = sklearn.model_selection.KFold(n_splits = 10)
random.shuffle(dataset_full)
kf.get_n_splits(dataset_full)
accuracy_list = []
recall_list = []
precision_list = []
f_list = []

k_list = [1000,750,500,250,150,100,75,50,25,10]
j=0

for train_index, test_index in kf.split(dataset_full):
    train_set_fold=[]
    test_set_fold=[]
    for i,instance in enumerate(dataset_full):
        if i in train_index:
            train_set_fold.append(instance)
        else:
            test_set_fold.append(instance)
            
    vocabulary_fold=get_vocabulary(train_set_fold, 1000)
    X_train_fold = [get_vector_text(vocabulary_fold, x[0]) for x in train_set_fold]
    Y_train_fold = [x[1] for x in train_set_fold]
    
    chi_x = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=k_list[j]).fit(X_train_fold, Y_train_fold)
    X_train_fold = chi_x.transform(X_train_fold)
    j = j+1
    
    svm_clf_fold=sklearn.svm.SVC(kernel='rbf', gamma='scale')
    svm_clf_fold.fit(X_train_fold, Y_train_fold)
    
    X_test_fold=[]
    Y_test_fold=[]
    
    for instance in test_set_fold:
        vector_instance=get_vector_text(vocabulary_fold,instance[0])
        X_test_fold.append(vector_instance)
        Y_test_fold.append(instance[1])
        
    Y_test_fold_gold=np.asarray(Y_test_fold)
    X_test_fold=np.asarray(chi_x.transform(X_test_fold))
    Y_test_predictions_fold=svm_clf_fold.predict(X_test_fold)

    accuracy_list.append(sklearn.metrics.accuracy_score(Y_test_fold_gold, Y_test_predictions_fold))
    recall_list.append(sklearn.metrics.recall_score(Y_test_fold_gold, Y_test_predictions_fold))
    precision_list.append(sklearn.metrics.precision_score(Y_test_fold_gold, Y_test_predictions_fold))
    f_list.append(sklearn.metrics.f1_score(Y_test_fold_gold, Y_test_predictions_fold))
    
    print ("Fold completed with a_score=",str(sklearn.metrics.accuracy_score(Y_test_fold_gold, Y_test_predictions_fold)))
    

print("Accuracy scores")
print(accuracy_list)
print("Recall scores")
print(recall_list)
print("Precision scores")
print(precision_list)
print("F1 scores")
print(f_list)

Fold completed with a_score= 0.8133333333333334
Fold completed with a_score= 0.7988888888888889
Fold completed with a_score= 0.79
Fold completed with a_score= 0.7733333333333333
Fold completed with a_score= 0.7588888888888888
Fold completed with a_score= 0.7722222222222223
Fold completed with a_score= 0.7633333333333333
Fold completed with a_score= 0.7566666666666667
Fold completed with a_score= 0.7488888888888889
Fold completed with a_score= 0.7188888888888889
Accuracy scores
[0.8133333333333334, 0.7988888888888889, 0.79, 0.7733333333333333, 0.7588888888888888, 0.7722222222222223, 0.7633333333333333, 0.7566666666666667, 0.7488888888888889, 0.7188888888888889]
Recall scores
[0.7318435754189944, 0.6789772727272727, 0.6717948717948717, 0.6897435897435897, 0.6305555555555555, 0.6717557251908397, 0.6568627450980392, 0.6135135135135135, 0.5520833333333334, 0.4497354497354497]
Precision scores
[0.7844311377245509, 0.7785016286644951, 0.8111455108359134, 0.7642045454545454, 0.729903536977492,

In [52]:
for x in f_list:
    print(x)
    

0.7572254335260115
0.7253414264036419
0.7349228611500702
0.7250673854447439
0.676602086438152
0.7203274215552524
0.7156208277703604
0.674591381872214
0.6523076923076924
0.5733558178752107
